In [ ]:
#!uv add loguru pydantic rich tenacity

<img src="./docs/parallelization1.jpg" alt="Parllelizaation 1" width="1200"/>
<img src="./docs/parallelization2.jpg" alt="Parllelizaation 2" width="1200"/>

In [ ]:
import asyncio
import os

from agents import ItemHelpers, Runner, RunResult, trace
from build_agents import ag_coding_explainer, ag_explanation_picker
from rich.console import Console
from rich.markdown import Markdown

# from IPython.display import Markdown, display

console = Console()

# print(f"INFO | Current working directory: {os.getcwd()}")
# print(f"INFO | Available LLMs: {llm_manager._registry.keys()}")
# print(f"INFO | Loading .env file success: {load_dotenv(override=True)}")
# print(f"INFO | Input CurriculumCheckOutput: {CurriculumCheckOutput.__dir__(BaseModel)}")

code_snippet = "Write a Python function called group_anagrams that takes a list of strings and groups the anagrams together. "

# Run three parallel explanation attempts
with trace(workflow_name="Parallelization"):
    res_1, res_2, res_3 = await asyncio.gather(
        Runner.run(starting_agent=ag_coding_explainer, input=code_snippet),
        Runner.run(starting_agent=ag_coding_explainer, input=code_snippet),
        Runner.run(starting_agent=ag_coding_explainer, input=code_snippet),
    )

    explanations: list[str] = [
        ItemHelpers.text_message_outputs(items=res_1.new_items),
        ItemHelpers.text_message_outputs(items=res_2.new_items),
        ItemHelpers.text_message_outputs(items=res_3.new_items),
    ]

    all_explanations: str = "\n\n".join(explanations)
    console.print(
        Markdown(markup=f"\n\nCandidate Explanations:\n\n{all_explanations}")
    )

    # Select the best explanation
    best_explanation: RunResult = await Runner.run(
        starting_agent=ag_explanation_picker,
        input=f"Code:\n{code_snippet}\n\nExplanations:\n{all_explanations}",
    )
print("\n\n-----")
console.print(
    Markdown(markup=f"Best explanation:\n{best_explanation.final_output}")
)
